# Working with JSON exercise
****
+ data source: http://jsonstudio.com/resources/
****

****
## JSON exercise

Using project's funded by the World Bank data set
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## imports for Python, Pandas

In [2]:
import pandas as pd
import json
from pandas.io.json import json_normalize


In [3]:
# import json data to a pandas dataframe
# change the data_file location to where the file exists on your computer
data_file = r'C:\Users\aregel\Documents\springboard\springboard_miniprojects\JSON_exercise\data\world_bank_projects.json'
raw_data = pd.read_json(data_file)

#### * In order to understand the data a little better, I will determine the time range the data covers and try to identify and time gaps

In [3]:
print(raw_data.approvalfy.max())
print(raw_data.approvalfy.min())
print(raw_data.approvalfy.value_counts())

2015
1999
2013    432
2014     66
2015      1
1999      1
Name: approvalfy, dtype: int64


#### * 84% of this data is from 2013.  There is not enough records for 1999 or 2015 to be represenative of those years.  Therefore, I am going to exclude the data from 1999 and 2015, and classify the date range as 2013-2014.

In [4]:
# create dataframe using boolean masks that excludes 1999 and 2015 data
data = raw_data[raw_data.approvalfy <= 2014]
data = data[data.approvalfy >= 2013]

### All of the following is data analysis on  projects funded by the World Bank from 2013-2014 (data source: http://jsonstudio.com/resources/ )

#### * To find the 10 countries with the most projects I will group data by countryname, find the number of times that name occurs from the size of that group, sort by the value of that size in descending order, then index only the first 10 rows and print the results *

In [5]:
# the 10 countries with the most projects
print(data.groupby(['countryname']).size().sort_values(ascending=False)[0:11])

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Nepal                              12
Kingdom of Morocco                 12
Republic of Mozambique             11
Africa                             11
Islamic Republic of Pakistan        9
dtype: int64


#### * To find the top major project theme codes I will first create a dictionary (theme_count) with the key as the theme code, and the value as the number of times that theme code occures. I then use that dictionary to create an ordered list of tuples that contain the number of times the theme occures and the unix code for the theme. Lastly, I map the theme name to the unix code, and print the top 10 themes. *

In [6]:
# create dictionary (theme_count) with the key, value as the theme code, # of occurrences
# create a list of code numbers and names
theme_count = {}
name_code = []
for lst in data.mjtheme_namecode:
    elems = len(lst)
    for elem in range(elems):
        code_num = lst[elem]['code']
        name = lst[elem]['name']
        name_code.append((code_num, name))
        theme_count[code_num] = theme_count.get(code_num,0) + 1
   

In [7]:
# loop over the theme_count dict to create a list of tuples, sort the tuples, index first 10
top_themecodes = sorted([(v,k) for k,v in theme_count.iteritems()], reverse=True)[0:10]
print(top_themecodes)

[(249, u'11'), (216, u'10'), (209, u'8'), (199, u'2'), (167, u'6'), (146, u'4'), (130, u'7'), (77, u'5'), (50, u'9'), (37, u'1')]


In [8]:
# create a dictionary to map the theme code to the theme name
name_map = {}
for code, name in name_code:
    if len(name) > 1:
        name_map[code] = name
print(name_map)

{u'11': u'Environment and natural resources management', u'10': u'Rural development', u'1': u'Economic management', u'3': u'Rule of law', u'2': u'Public sector governance', u'5': u'Trade and integration', u'4': u'Financial and private sector development', u'7': u'Social dev/gender/inclusion', u'6': u'Social protection and risk management', u'9': u'Urban development', u'8': u'Human development'}


In [9]:
# use the name_map dictionary to map the theme name to the theme code 
print("The top 10 major project themes:")
for num, code in top_themecodes:
    print(name_map[code])

The top 10 major project themes:
Environment and natural resources management
Rural development
Human development
Public sector governance
Social protection and risk management
Financial and private sector development
Social dev/gender/inclusion
Trade and integration
Urban development
Economic management


#### * To create a dataframe with the missing names filled in, I updated the missing values using the name_map dictionary

In [10]:
# update the Data dataframe so that no project names are missing

for lst in data.mjtheme_namecode: # accesses the list of dictionaries
    elems = len(lst)              # finds how many dictionaries are in each list
    for num in range(0,elems):    # accesses each dictionary using the list index
        key = lst[num]['code']    # accesses the code number
        lst[num]['name'] = name_map[key] # replaces the name using the code number and name_map

In [11]:
print(data.mjtheme_namecode.head(30))

2     [{u'code': u'5', u'name': u'Trade and integrat...
3     [{u'code': u'7', u'name': u'Social dev/gender/...
4     [{u'code': u'5', u'name': u'Trade and integrat...
5     [{u'code': u'6', u'name': u'Social protection ...
6     [{u'code': u'2', u'name': u'Public sector gove...
7     [{u'code': u'11', u'name': u'Environment and n...
8     [{u'code': u'10', u'name': u'Rural development...
9     [{u'code': u'2', u'name': u'Public sector gove...
10    [{u'code': u'10', u'name': u'Rural development...
11    [{u'code': u'10', u'name': u'Rural development...
12    [{u'code': u'4', u'name': u'Financial and priv...
13    [{u'code': u'5', u'name': u'Trade and integrat...
14    [{u'code': u'6', u'name': u'Social protection ...
15    [{u'code': u'10', u'name': u'Rural development...
16    [{u'code': u'10', u'name': u'Rural development...
17    [{u'code': u'8', u'name': u'Human development'...
18    [{u'code': u'8', u'name': u'Human development'...
19    [{u'code': u'2', u'name': u'Public sector 